## Setup

If you haven't already, install the toolkit and dependencies using the [Setup](./00-Setup.ipynb) notebook.

## Introduction

The [Model Context Protocol](https://modelcontextprotocol.io/introduction) (MCP) is an open protocol that standardizes how applications provide context to LLMs.

In this example, we're going to create a 'catalog' of tools, one per tenant in a [multi-tenant](https://github.com/awslabs/graphrag-toolkit/blob/main/docs/lexical-graph/multi-tenancy.md) graph. Each tool is capable of answering domain-specific questions based on the data in its tenant graph. This catalog will be advertised to clients via an MCP server. Clients (typically agents and LLMs) can then browse the catalog and choose appropriate tools for addressing their information goals.

Each tool in the catalog is accompanied by an auto-generated description that helps a client understand the domain, scope, potential uses and kinds of questions covered by the tool. The catalog also includes a 'search' tool, which, given the name of an entity or concept, recommends one or more domain tools with knowledge of the search term.

### Additional setup

#### Create tenant-specific lexical graphs

Besides the initial setup described above, this example requires you to have created two tenant-specific lexical graphs: a Neptune documentation lexical graph (installed in the _default_ tenant graph), and an Amazon OpenSearch documentation lexical graph (installed in the `aoss` tenant graph):

  - To create the Neptune documentation lexical graph, run either notebook [01-Combined-Extract-and-Build](./01-Combined-Extract-and-Build.ipynb) or notebook [02-Separate-Extract-and-Build](./02-Separate-Extract-and-Build.ipynb).
  - To create the Amazon OpenSearch documentation lexical graph, run the create cell in notebook [05-Multi-Tenancy](./05-Multi-Tenancy.ipynb)

#### Provide Bedrock model access for Claude 3.7 Sonnet

Ensure you have [requested access](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access-modify.html) in Amazon Bedrock to the `anthropic.claude-3-7-sonnet-20250219-v1:0` foundation model.

#### Install additional dependencies

The last thing you need to do is install these additional dependencies:

In [ ]:
!pip install fastmcp strands-agents

## Create an MCP server

The following cell creates an MCP server that hosts a catalog of tools – one per tenant graph. The cell takes a few seconds to run while the tool descriptions are auto-generated.

In [ ]:
import logging

from graphrag_toolkit.lexical_graph import set_advanced_logging_config

set_advanced_logging_config(
    logging_level=logging.DEBUG,
    included_modules={
        logging.DEBUG: [
            'graphrag_toolkit.lexical_graph.protocols', 
            'graphrag_toolkit.lexical_graph.retrieval.summary'
        ],
        logging.INFO: '*',
    },
    excluded_modules={
        logging.DEBUG: ['opensearch', 'boto', 'urllib'],
        logging.INFO: ['opensearch', 'boto', 'urllib', 'mcp', 'httpx'],
    }
)

In [ ]:
%reload_ext dotenv
%dotenv

import os

from graphrag_toolkit.lexical_graph.storage import GraphStoreFactory, VectorStoreFactory
from graphrag_toolkit.lexical_graph.protocols import create_mcp_server

graph_store = GraphStoreFactory.for_graph_store(os.environ['GRAPH_STORE'])
vector_store = VectorStoreFactory.for_vector_store(os.environ['VECTOR_STORE'])

mcp_server = create_mcp_server(graph_store, vector_store)

print('Server initialized')

### Start the server

The cell below starts the MCP server using the Streamable HTTP transport on a background thread.

In [ ]:
import threading

def run_server():
    mcp_server.run(transport='streamable-http', log_level='warning')
    
thread = threading.Thread(target=run_server)
thread.start()

## Create an MCP client and AI agent

[Strands Agents](https://strandsagents.com/latest/) is an open source SDK that takes a model-driven approach to building and running AI agents in just a few lines of code.

In the cell below we create an MCP client that we can then use in a Strands Agent for answering cross-domain questions.

In [ ]:
from mcp.client.streamable_http import streamablehttp_client
from strands.tools.mcp.mcp_client import MCPClient

def create_streamable_http_transport():
    return streamablehttp_client('http://localhost:8000/mcp/')

mcp_client = MCPClient(create_streamable_http_transport)

### Inspect the tool descriptions

The code below prints out the tool descriptions that have been auto-generated from the tenant graphs. Each tool is named after its tenant. The tool for the default tenant graph is named `default_`. Note that besides the tenant-specific tools, there is also a `search_` tool.

In [ ]:
from strands import Agent

with mcp_client:
    
    tools = mcp_client.list_tools_sync()
    
    for tool in tools:
        print(f"{tool.tool_spec['name']}: {tool.tool_spec['description']}")
        print('\n-------------------------------------\n')

### Create an agent and ask a question

We can now create a Strands AI Agent, and ask a question. The agent will choose the most appropriate tools for answering the question.

In [ ]:
with mcp_client:

    tools = mcp_client.list_tools_sync()
    agent = Agent(tools=tools)
    
    response = agent("What are the differences between Amazon Neptune and Amazon OpenSearch Serverless?")